<a href="https://colab.research.google.com/github/LudwigBitHoven/MillionAgents_testTask/blob/main/scrapers/metro_webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установим selenium

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


# Импортируем и настроим джентльменский набор для скрапинга

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json
import pandas as pd
import requests

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()
# аргумент для работы с мобильной версией сайта
# options.add_argument('--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1')
options.add_argument("start-maximized")
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

# Посмотрим что на странице

Страница товара не имеет бесконечного скролла и товары распределены по отдельным url-ам вида



> "https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page="


#### Можно захардкодить скрапируемые адреса с товарами следующим образом, меняя лишь параметр page:


In [ ]:
pages = []
for i in range(1, 17):
  pages.append(f"https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page={i}")
pages

['https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=1',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=2',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=3',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=4',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=5',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=6',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=7',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=8',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=9',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=10',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=11',
 'https://online.metro-cc.ru/category/chaj-kofe-kaka

## А можно сделать лучше и спарсить количество страниц по указанному адресу категории товаров:

In [ ]:
# категория "кофе"
url = "https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=1"
driver.get(url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html)
# получаем все связанное с пагинацией
pagination = soup.find("ul", class_="catalog-paginate v-pagination")
list(pagination)

[<li data-v-46976e4c=""><a aria-current="true" class="v-pagination__item catalog-paginate__item nuxt-link-active v-pagination__item--active" data-v-46976e4c="" href="/category/chaj-kofe-kakao/kofe?from=under_search">1</a></li>,
 <li data-v-46976e4c=""><a class="v-pagination__item catalog-paginate__item nuxt-link-active" data-v-46976e4c="" href="/category/chaj-kofe-kakao/kofe?from=under_search&amp;page=2">2</a></li>,
 <li data-v-46976e4c=""><a class="v-pagination__item catalog-paginate__item nuxt-link-active" data-v-46976e4c="" href="/category/chaj-kofe-kakao/kofe?from=under_search&amp;page=3">3</a></li>,
 <li data-v-46976e4c=""><span class="v-pagination__more catalog-paginate__item disabled dots" data-v-46976e4c="">...</span></li>,
 <li data-v-46976e4c=""><a class="v-pagination__item catalog-paginate__item nuxt-link-active" data-v-46976e4c="" href="/category/chaj-kofe-kakao/kofe?from=under_search&amp;page=15">15</a></li>,
 <li data-v-46976e4c=""><a class="v-pagination__item catalog-pag

In [ ]:
list(pagination)[-2] # кнопка, ведущая на самую последнюю страницу в категории

<li data-v-46976e4c=""><a class="v-pagination__item catalog-paginate__item nuxt-link-active" data-v-46976e4c="" href="/category/chaj-kofe-kakao/kofe?from=under_search&amp;page=17">17</a></li>

In [ ]:
last_page = int(list(pagination)[-2].text)
last_page # индекс последней страницы из имени кнопки

17

Теперь получим пул url адресов, с которых необходимо будет соскрапить товары

In [ ]:
pages = []
for i in range(1, last_page + 1):
  pages.append(f"https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page={i}")
pages

['https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=1',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=2',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=3',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=4',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=5',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=6',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=7',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=8',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=9',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=10',
 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=11',
 'https://online.metro-cc.ru/category/chaj-kofe-kaka

## Попробуем вытянуть данные о товаре со страницы

### Соскрапим все простое:

In [ ]:
driver.get("https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=10") # возьмем страницу
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html)

Выделяем название товара

In [ ]:
products = soup.findAll("div", class_="product-card-photo catalog-2-level-product-card__photo style--catalog-2-level-product-card")
products[0].findAll("a")[0]["title"] # выделяем название товара

'Кофе Жокей классический 3в1 растворимый (12г x 50шт), 600г'

Выделяем бренд в отдельном запросе

In [ ]:
products[0].findAll("a")[0]["href"] # ссылка на товар без базового url
url = "https://online.metro-cc.ru" + products[0].findAll("a")[0]["href"]
subsoup = BeautifulSoup(requests.get(url).text, 'html.parser')
brand = subsoup.findAll("a", class_="product-attributes__list-item-link reset-link active-blue-text")[-3]
brand = brand.text.replace("\n", "") # удаляем переносы строки
brand = brand.replace(" ", "") # удаляем пробелы
brand

'ЖОКЕЙ'

Выделяем id товара

In [ ]:
products = soup.findAll("div", class_="catalog-2-level-product-card product-card subcategory-or-type__products-item with-prices-drop")
int(products[0]["data-sku"]) # id товара

234805

Напишем функцию для определения бренда

In [ ]:
def get_brand(href):
  """
    Функция выполняет дополнительный запрос на страницу товара, чтобы
    вытянуть название бренда
  """
  try:
    url = "https://online.metro-cc.ru" + href
    subsoup = BeautifulSoup(requests.get(url, timeout=6).text, 'html.parser')
    brand = subsoup.findAll("a", class_="product-attributes__list-item-link reset-link active-blue-text")[-3]
    brand = brand.text.replace("\n", "")
    brand = brand.replace(" ", "")
    return brand
  except Exception: # хорошо бы исправить в дальнейшем
    print(f"Error occured at {url}")
    return ""

get_brand(products[0].find("a")["href"]) # протестируем функцию скрапа бренда

'MACCOFFEE'

### Цены на сайте можно разделить на две категории: цены в онлайн магазине и цены в торговом центре. Один товар может иметь один ценник в онлайн магазине. В свою очередь ценник может иметь два значения (акционная цена и обычная цена), или одно (обычная цена). Ценник в торговом центре может иметь неограниченное количество значений для каждой из действующих в центре акций. Например один товар может проходить по большому количеству акций и иметь разные цены в зависимости от приобретаемого количества этого товара:


*   699р ~~899р~~ - 1 шт
*   1199р ~~1799р~~ - 2 шт
*   1699р ~~1999р~~ - 3 шт и т.д.

### Также возможен вариант, когда форматы цен чередуются:


*   699р ~~899р~~ - 1 шт
*   1199р - 2 шт
*   1699р - 3 шт и т.д.

### Напишем функцию, чтобы собирать цены, учитывая вышеописанные нюансы:

In [ ]:
def get_prices_shop(product):
  res = []
  # ищем все значения в ценнике
  temp = product.findAll("li", class_="product-prices-lines__item")
  for i in temp:
    offers = i.findAll("span", class_="product-prices-lines__item-count nowrap") # находим условия
    prices = i.find("span", class_="product-price__sum-rubles") # находим цену по акции
    for idx, price in enumerate(prices):
      price = price.text.replace("\xa0", "")
      offer = offers[idx - 1].text.strip()
      res.append((int(price), offer))
  return res

def get_prices_online(product):
  res = {
    "onlineDiscount": 0, # онлайн заказ по скидке
    "onlineRegular": 0, # онлайн заказ без скидки
  }
  # находим ценник
  temp = product.find("div", class_="product-unit-prices__trigger")
  temp = temp.findAll("span", class_="product-price__sum-rubles")
  temp = [int(i.text.replace("\xa0", "")) for i in temp]
  if len(temp) == 1:
    # если товар не акционный, то заполняем только onlineRegular
    res["onlineRegular"] = temp[0]
  else:
    # если товар акционный, то заполняем поле onlineDiscount
    res["onlineDiscount"] = temp[0]
    res["onlineRegular"] = temp[1]
  return res


products = soup.findAll("div", class_="catalog-2-level-product-card product-card subcategory-or-type__products-item with-prices-drop")
for i in products:
  button_txt = i.find("span", class_="simple-button__text").text
  if "В корзину" in button_txt or "Только" in button_txt: # проверка на наличие в магазине
    print(get_prices_shop(i), get_prices_online(i))

[(429, 'от 1 шт')] {'onlineDiscount': 449, 'onlineRegular': 529}
[(289, 'от 1 шт')] {'onlineDiscount': 299, 'onlineRegular': 499}
[(389, 'от 1 шт')] {'onlineDiscount': 419, 'onlineRegular': 469}
[(389, 'от 1 шт')] {'onlineDiscount': 399, 'onlineRegular': 469}
[(359, 'от 1 шт')] {'onlineDiscount': 399, 'onlineRegular': 449}
[(229, 'от 1 шт')] {'onlineDiscount': 229, 'onlineRegular': 319}
[(349, 'от 1 шт')] {'onlineDiscount': 349, 'onlineRegular': 419}
[(149, 'от 1 шт')] {'onlineDiscount': 149, 'onlineRegular': 207}
[(319, 'от 1 шт')] {'onlineDiscount': 349, 'onlineRegular': 389}
[(349, 'от 1 шт')] {'onlineDiscount': 379, 'onlineRegular': 399}
[(349, 'от 1 шт')] {'onlineDiscount': 349, 'onlineRegular': 359}
[(329, 'от 1 шт')] {'onlineDiscount': 339, 'onlineRegular': 359}
[(359, 'от 1 шт')] {'onlineDiscount': 0, 'onlineRegular': 359}
[(349, 'от 1 шт')] {'onlineDiscount': 349, 'onlineRegular': 399}
[(299, 'от 1 шт')] {'onlineDiscount': 299, 'onlineRegular': 369}
[(299, 'от 1 шт')] {'online

In [ ]:
products = soup.findAll("div", class_="catalog-2-level-product-card product-card subcategory-or-type__products-item with-prices-drop")
tup = {
          "product_id": int(products[0]["data-sku"]),
          **get_prices_online(products[0]),
          "prices_shop": get_prices_shop(products[0])
      }
temp = products[0].findAll("div", class_="product-card-photo catalog-2-level-product-card__photo style--catalog-2-level-product-card")
tup["title"] = temp[0].find("a")["title"] # название продукта
tup["link"] = temp[0].find("a")["href"] # ссылка на товар
tup["brand"] = get_brand(tup["link"]) # бренд

for line in tup:
  print(line, tup[line])

product_id 234805
onlineDiscount 449
onlineRegular 529
prices_shop [(429, 'от 1 шт')]
title Кофе Жокей классический 3в1 растворимый (12г x 50шт), 600г
link /products/kofe-zhokey-klassicheskiy-3v1-rastvorimyy-12g-x-50sht-600g
brand ЖОКЕЙ


# Скрапинг

Проверим для какого магазина собирается информация:

In [ ]:
soup.find("address", class_="header-address__receive-address header-address__receive-address--blured").text.strip()

'Москва, Ул. Ленинградское шоссе, д. 71Г (м. Речной вокзал)'

In [509]:
data = {"product_id": [], "title": [], "link": [], "onlineRegular": [], "onlineDiscount": [], "prices_shop": [], "brand": []}

cnt = 0
for page in pages:
  driver.get(page) # делаем запрос и получаем страницу
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # скроллим вниз для загрузки всех товаров
  time.sleep(3) # ждем прогрузки
  html = driver.page_source
  soup = BeautifulSoup(html)
  # составляем список товаров
  products = soup.findAll("div", class_="catalog-2-level-product-card product-card subcategory-or-type__products-item with-prices-drop")
  for product in products:
    button_txt = product.find("span", class_="simple-button__text").text
    if "В корзину" in button_txt or "Только" in button_txt: # проверка на наличие в магазине
      time.sleep(1)
      tup = {
          "product_id": int(product["data-sku"]),
          **get_prices_online(product),
          "prices_shop": get_prices_shop(product)
      }
      temp = product.findAll("div", class_="product-card-photo catalog-2-level-product-card__photo style--catalog-2-level-product-card")
      tup["title"] = temp[0].find("a")["title"] # название продукта
      tup["link"] = temp[0].find("a")["href"] # ссылка на товар
      tup["brand"] = get_brand(tup["link"]) # бренд
      for key in tup:
        data[key].append(tup[key])
        cnt += 1
  print(f"Page '{page}' scraped")
print(f"Scraped all data, totally {cnt}")

https://online.metro-cc.ru/products/kofe-alta-roma-espresso-v-zernakh-1kg
Page 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=1' scraped
https://online.metro-cc.ru/products/molotyj-dallmayr-efiopiya-500g
Page 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=2' scraped
Page 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=3' scraped
https://online.metro-cc.ru/products/ambassador-naturalnyj-zharenyj-platinum-1kg
Page 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=4' scraped
Page 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=5' scraped
Page 'https://online.metro-cc.ru/category/chaj-kofe-kakao/kofe?from=under_search&page=6' scraped
https://online.metro-cc.ru/products/v-zernah-carraro-primo-mattino-1-kg
https://online.metro-cc.ru/products/nescafe-sublimirovannyj-gold-190g
Page 'https://online.metro-cc.ru/category/chaj-kofe-kak

# Сохраняем датафрейм

In [511]:
df = pd.DataFrame.from_dict(data)
df

,product_id,title,link,onlineRegular,onlineDiscount,prices_shop,brand
0,642122,Кофе Nescafe Gold растворимый сублимированный ...,/products/nescafe-s-dobavleniem-naturalnogo-zh...,2399,1599,"[(1599, от 1 шт)]",NESCAF
1,122743,Кофе Nescafe Gold растворимый сублимированный ...,/products/kofe-nescafe-gold-rastvorimiy-sublim...,2269,1899,"[(1799, от 1 шт)]",NESCAF
2,407889,"Кофе Bushido Original растворимый, 100г",/products/bushido-sublimirovannyj-original-100g,899,649,"[(589, от 1 шт), (535, от 6 шт)]",BUSHIDO
3,287768,RIOBA Кофе в зернах натуральный жареный 100% а...,/products/rioba-v-zernah-naturalnyj-zharenyj-1...,1529,1199,"[(1199, от 1 шт)]",RIOBA
4,580564,Кофе Egoiste Arabica Premium натуральный жарен...,/products/egoiste-v-zernah-naturalnyj-zharenyj...,2099,1499,"[(1599, от 1 шт)]",EGOISTE
...,...,...,...,...,...,...,...
388,117382,Кофе в капсулах L’or Espresso Ristretto для ко...,/products/52g-l-rsquoor-ristretto,3990,0,"[(3190, от 1 уп)]",L'OR
389,117387,Набор кофе в капсулах Coffesso для кофемашин N...,/products/50h5g-coffesso-nabor,6594,0,"[(5214, от 1 уп)]",COFFESSO
390,247918,Кофе Poetti Professional Aroma Vending зерново...,/products/1kg-kofe-poetti-professional-aroma-v...,1279,999,"[(999, от 1 шт)]",POETTI
391,117381,Кофе в капсулах L’or Espresso Splendente для к...,/products/52g-l-rsquoor-espr-splendent,3990,0,"[(3190, от 1 уп)]",L'OR


In [512]:
df.to_csv("products.csv")